In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
    
df = pd.read_csv('../../data/imp_exp_prod_flat_all.csv')
df.drop(columns='Row Number', inplace=True)
df = df[['Area', 'Item', 'Year', 'Production', 'Import Quantity', 'Export Quantity']]
df.rename(columns={'Area': 'Country', 'Item':'Produce', 'Production':'Production Quantity'}, inplace=True)
df.head()

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
0,Armenia,Potatoes,2000,290260.0,390.0,300.0
1,Armenia,Potatoes,2001,363834.0,2290.0,0.0
2,Armenia,Potatoes,2002,374263.0,1918.0,0.0
3,Armenia,Potatoes,2003,507518.0,2853.0,7.0
4,Armenia,Potatoes,2004,576427.0,1553.0,0.0


In [2]:
# create a lookup table for countries and their centroid locations
lookup = pd.read_csv('../../data/country_centroids.csv')
lookup.rename(columns={'country':'code'}, inplace=True)
lookup.index = lookup['name']
(lookup.drop(columns=['name'])).to_csv('../../data/countries_lookup.csv')


# FIXME --------------------------------------------------------------------------
# ********************************************************************************
# Assuming that China is a sum of mainland, taiwan, hong kong and macao
# 'Serbia and Montenegro' - set coordinates to average of Serbia and Montenegro
# 'South Sudan' and 'Sudan (former)'- set coordinates to Sudan's coordinates

# print regions that are not present in the countries lookup table
set(df[~df['Country'].isin(lookup['name'])]['Country'])

{'Africa',
 'Americas',
 'Asia',
 'Australia & New Zealand',
 'Caribbean',
 'Central America',
 'Central Asia',
 'China',
 'Eastern Africa',
 'Eastern Asia',
 'Eastern Europe',
 'Europe',
 'European Union',
 'Land Locked Developing Countries',
 'Least Developed Countries',
 'Low Income Food Deficit Countries',
 'Melanesia',
 'Middle Africa',
 'Net Food Importing Developing Countries',
 'Northern Africa',
 'Northern America',
 'Northern Europe',
 'Occupied Palestinian Territory',
 'Oceania',
 'Polynesia',
 'Small Island Developing States',
 'South America',
 'South-Eastern Asia',
 'Southern Africa',
 'Southern Asia',
 'Southern Europe',
 'Western Africa',
 'Western Asia',
 'Western Europe',
 'World'}

In [3]:
# remove the regions printed above from the dataframe
df = df[df['Country'].isin(lookup.index)]

# randomly print 10 entries from the dataframe for an example of what it contains
df.sample(n=10)

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
145569,Estonia,Cabbages and other brassicas,2016,18977.0,4414.0,80.0
3626,Iran (Islamic Republic of),Lentils,2014,71507.0,44934.0,428.0
159310,Croatia,Flax fibre and tow,2005,0.0,0.0,0.0
70036,Uruguay,Grapes,2008,112883.0,102.0,11.0
157410,Hungary,"Roots and tubers, nes",2016,0.0,43.0,3.0
60954,Thailand,Grapes,2005,68468.0,11481.0,38.0
1903,India,"Pumpkins, squash and gourds",2007,4259627.0,0.0,9430.0
151863,Ghana,"Fruit, fresh nes",2001,44144.0,3.0,3.0
54944,Eswatini,Cotton lint,2001,3000.0,6004.0,1010.0
1740,India,Rapeseed,2004,6291400.0,2274.0,16119.0


In [4]:
def getMaximumQuantity(selectedData):
    largestProduced = int(selectedData.nlargest(1, 'Production Quantity')['Production Quantity'])
    largestImported = int(selectedData.nlargest(1, 'Import Quantity')['Import Quantity'])
    largestExported = int(selectedData.nlargest(1, 'Export Quantity')['Export Quantity'])
    return max([largestProduced, largestImported, largestExported])
    
# calculate top 10 produced, imported and exported crops for each country in each year
top10_statistics = {}
top10_statistics['countries'] = countrySet = sorted(set(df['Country']))

for country in countrySet: 
    # log available years for the given produce
    yearStatistics = {}
    years = sorted(set(df[df['Country'] == country]['Year']))
    yearStatistics['available_years'] = years
    
    # log the largest quantity across all years
    yearStatistics['largest_quantity'] = getMaximumQuantity(df[df['Country'] == country])
    
    for year in years:
        selected = df[(df['Country'] == country) & (df['Year'] == year)]
        
        # log top 10 produced, imported, and exported crops, plus the largest quantity for each year
        topProduce = selected.nlargest(10, 'Production Quantity')
        topImports = selected.nlargest(10, 'Import Quantity')
        topExports = selected.nlargest(10, 'Export Quantity')
        yearStatistics[year] = {
            'largest_quantity': getMaximumQuantity(selected),
            'Production': {
                'produce': list(topProduce['Produce']),
                'production': list(topProduce['Production Quantity']),
                'import': list(topProduce['Import Quantity']),
                'export': list(topProduce['Export Quantity'])
            },
            'Imports': {
                'produce': list(topImports['Produce']),
                'production': list(topImports['Production Quantity']),
                'import': list(topImports['Import Quantity']),
                'export': list(topImports['Export Quantity'])
            },
            'Exports': {
                'produce': list(topExports['Produce']),
                'production': list(topExports['Production Quantity']),
                'import': list(topExports['Import Quantity']),
                'export': list(topExports['Export Quantity'])
            }
        }
        
    top10_statistics[country] = yearStatistics

In [5]:
# dump top 10 statistics to a json file
json.dump(top10_statistics, open('../../data/top10_crops.json', 'w'))